In [1]:
import transcriptic 
from transcriptic import commands, Container
from transcriptic.config import Connection
from transcriptic.jupyter import objects
import json
from pysd2cat.data import tx_fcs
from pysd2cat.data import pipeline
from pysd2cat.analysis import biofab_live_dead_analysis as blda
from pysd2cat.data import biofab_live_dead as bld


from os.path import expanduser
import requests
import os
import zipfile
import shutil
import fnmatch
import glob
import pandas as pd
import ast

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

Connection.from_file("~/.transcriptic")

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
#protocol = objects.protocol('pr1drp6hzrchh79')
[x['name'] for x in transcriptic.api.get_protocols() if x['id'] == 'pr1drp6hzrchh79']

[]

In [2]:
container_string = '423021.3' #'422936' #
results = transcriptic.api.inventory(container_string)


In [4]:
results

{'results': [{'id': 'ct1dqvp6nd7gej7',
   'container_type_id': '96-pcr',
   'barcode': '58304253',
   'deleted_at': None,
   'created_at': '2019-10-30T13:28:25.662-07:00',
   'organization_id': 'org1amxh23ednpz',
   'slot': None,
   'cover': None,
   'test_mode': False,
   'label': '423021.1',
   'location_id': 'loc1b68jjfjhatk',
   'shipment_id': 'sr1dqvp6p9z9h9p',
   'kit_request_id': None,
   'storage_condition': 'cold_80',
   'shipment_code': 'RTZ',
   'status': 'available',
   'expires_at': None,
   'aliquot_count': 93,
   'container_type': {'id': '96-pcr',
    'name': '96-Well twin.tec PCR Plate',
    'well_count': 96,
    'well_depth_mm': '14.6',
    'well_volume_ul': '160.0',
    'capabilities': ['magnetic_separate',
     'thermocycle',
     'qrt_thermocycle',
     'spin',
     'incubate',
     'seal',
     'image_plate',
     'sanger_sequence',
     'miniprep_destination',
     'stamp',
     'flash_freeze',
     'echo_dest',
     'magnetic_transfer_pcr',
     'bluewash',
     

In [5]:
[x['id'] for x in results['results']]

['ct1dqvp6nd7gej7', 'ct1dqvp6nm67z9b']

In [3]:
c = objects.Container(results['results'][0]['id'])

In [6]:
#c.aliquots.loc[c.aliquots['Name'] == 'UWBF_25784']
container_strains = c.aliquots.Name.unique()
sample_strains = set([ 'UWBF_24952', 'UWBF_24959', 'UWBF_24960',
       'UWBF_24961', 'UWBF_24962', 'UWBF_25784', 'UWBF_24864',
       'UWBF_23970', 'UWBF_6390'])
sample_strains.issubset(container_strains)
#col = 1
#c.aliquots.loc[(c.aliquots.index + col) % 12  == 0]

True

In [123]:
def get_matching_aliquots(strains, container_id, merge_key="Name"):
    c = objects.Container(container_id)
    strain_df = pd.DataFrame({merge_key : strains})
    c.aliquots['index'] = c.aliquots.index
    matches = c.aliquots.merge(strain_df, on=merge_key, how="inner")
    matches = matches.set_index('index')
    
    print("Matched:\n", str(matches.groupby(["Name"]).agg(len)))
    print("Didn't Match:\n", str([x for x in strains if x not in matches["Name"].unique()]))
    return matches
    
def get_src_wells_from_aliquots(container_id, matching_aliquots):
    print(matching_aliquots)
    return [ "{}/{}".format(container_id, x) for x in matching_aliquots.index.tolist() ]
    
def get_src_wells_matching_aliquots(strains, container_search_string=""):
    results = transcriptic.api.inventory(container_search_string)
    for result in results['results']:
        container_id = result['id']
        matching_aliquots = get_matching_aliquots(strains, container_id)
        if matching_aliquots is not None:
            src_wells = get_src_wells_from_aliquots(container_id, matching_aliquots)
            return src_wells
    return None
    


strains = [ 'UWBF_24926', 'UWBF_24952', 'UWBF_24959', 'UWBF_24960', 'UWBF_24961', 'UWBF_24962', 'UWBF_25784', 'UWBF_23970', 'UWBF_6390', 'UWBF_24864' ]
#strains = [  'UWBF_24952']

get_src_wells_matching_aliquots(strains, container_search_string=container_string)

Matched:
             Id  Volume  SBH_URI
Name                           
UWBF_23970  3   3       3      
UWBF_24864  3   3       3      
UWBF_24926  12  12      12     
UWBF_24952  12  12      12     
UWBF_24959  12  12      12     
UWBF_24960  12  12      12     
UWBF_24961  12  12      12     
UWBF_24962  12  12      12     
UWBF_25784  12  12      12     
UWBF_6390   3   3       3      
Didn't Match:
 []
             Name               Id         Volume                                             SBH_URI
index                                                                                                
0      UWBF_24926  aq1dqvp6ntrkkp5  40:microliter  https://hub.sd2e.org/user/sd2e/design/UWBF_24926/1
1      UWBF_24926  aq1dqvp6ntsqst6  40:microliter  https://hub.sd2e.org/user/sd2e/design/UWBF_24926/1
2      UWBF_24926  aq1dqvp6nttuyx7  40:microliter  https://hub.sd2e.org/user/sd2e/design/UWBF_24926/1
3      UWBF_24926  aq1dqvp6nttuyx8  40:microliter  https://hub.sd2e.org/user/s

['ct1dqvp6ntpb7e3/0',
 'ct1dqvp6ntpb7e3/1',
 'ct1dqvp6ntpb7e3/2',
 'ct1dqvp6ntpb7e3/3',
 'ct1dqvp6ntpb7e3/4',
 'ct1dqvp6ntpb7e3/5',
 'ct1dqvp6ntpb7e3/6',
 'ct1dqvp6ntpb7e3/7',
 'ct1dqvp6ntpb7e3/8',
 'ct1dqvp6ntpb7e3/9',
 'ct1dqvp6ntpb7e3/10',
 'ct1dqvp6ntpb7e3/11',
 'ct1dqvp6ntpb7e3/12',
 'ct1dqvp6ntpb7e3/13',
 'ct1dqvp6ntpb7e3/14',
 'ct1dqvp6ntpb7e3/15',
 'ct1dqvp6ntpb7e3/16',
 'ct1dqvp6ntpb7e3/17',
 'ct1dqvp6ntpb7e3/18',
 'ct1dqvp6ntpb7e3/19',
 'ct1dqvp6ntpb7e3/20',
 'ct1dqvp6ntpb7e3/21',
 'ct1dqvp6ntpb7e3/22',
 'ct1dqvp6ntpb7e3/23',
 'ct1dqvp6ntpb7e3/24',
 'ct1dqvp6ntpb7e3/25',
 'ct1dqvp6ntpb7e3/26',
 'ct1dqvp6ntpb7e3/27',
 'ct1dqvp6ntpb7e3/28',
 'ct1dqvp6ntpb7e3/29',
 'ct1dqvp6ntpb7e3/30',
 'ct1dqvp6ntpb7e3/31',
 'ct1dqvp6ntpb7e3/32',
 'ct1dqvp6ntpb7e3/33',
 'ct1dqvp6ntpb7e3/34',
 'ct1dqvp6ntpb7e3/35',
 'ct1dqvp6ntpb7e3/36',
 'ct1dqvp6ntpb7e3/37',
 'ct1dqvp6ntpb7e3/38',
 'ct1dqvp6ntpb7e3/39',
 'ct1dqvp6ntpb7e3/40',
 'ct1dqvp6ntpb7e3/41',
 'ct1dqvp6ntpb7e3/42',
 'ct1dqvp6ntpb7e3/43'

In [109]:
c = objects.Container("ct1dqvp6ntpb7e3")

In [115]:
c.

SyntaxError: invalid syntax (<ipython-input-115-cf463a5954f9>, line 1)

In [59]:
def get_design_df(design_file):
    j = json.load(open(design_file, 'r'))
    return pd.read_json(j['design'])
    
print(os.getcwd())
this_dir = os.getcwd()
#df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/YEAST_STATES_GROWTH_CURVE/experiments/growth_curve_test1/design_growth_curve_test1.json')))
#df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/YEAST_STATES_GROWTH_CURVE/experiments/growth_curve_test1_corrected/design_growth_curve_test1_corrected.json')))
#df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/YEAST_STATES/experiments/dose_response_test1/design_dose_response_test1.json')))
#df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/YEAST_STATES/experiments/dose_response_test1/design_dose_response_test1.json')))
#df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/YEAST_STATES/experiments/YeastSTATES CRISPR Long Duration Time Series Request (20191208)/design_YeastSTATES CRISPR Long Duration Time Series Request (20191208).json')))
df = get_design_df(os.path.realpath(os.path.join(this_dir, '../../../tmp/NOVEL_CHASSIS/experiments/CP_NovelChassis_OR_Circuit_Cycle0/design_CP_NovelChassis_OR_Circuit_Cycle0.json')))
df

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/pysd2cat/notebooks


,colony,column_id,cuminic_acid,induction_info.induction_reagents.ind_cult_vol,induction_info.induction_reagents.inducer,induction_info.induction_reagents.inducer_layout.value,induction_info.induction_reagents.inducer_solvent,induction_info.induction_reagents.induction_base_media,induction_info.induction_reagents.media_vol,induction_info.induction_time.induction_plate_info,induction_info.induction_time.sampling_time,induction_info.induction_time.total_induction_time,induction_info.sampling_info.flow_cult_vol,induction_info.sampling_info.flow_solv,induction_info.sampling_info.flow_solv_vol,induction_info.sampling_info.read_cult_vol,induction_info.sampling_info.read_solv,induction_info.sampling_info.read_solv_vol,induction_info.sampling_info.stain_before_flow,inoc_info.inc_time_1,inoc_info.inoc_media_vol,inoc_info.inoc_vol,inoc_info.inoculation_media,iptg,lab,plate_reader_info.fluor_em,plate_reader_info.fluor_ex,plate_reader_info.gain,protocol,recovery_info,recovery_media,replicate,run_info.only_endpoint_flow,run_info.read_each_induction,run_info.read_each_recovery,run_info.save_for_rnaseq,run_info.skip_first_flow,strain,temperature,total_induction_time,validate_samples,batch,id,experiment_id,measurement_type,timepoint,output_id,container
0,3,1,0.0,10:microliter,"{'containerId': 'ct1dt583rfgjx8p', 'wellIndex': 0}",select_cols,Water,SC Media,1000:microliter,same-induction-plate,30:minute,8:hour,100:microliter,None,0:microliter,100:microliter,None,0:microliter,Sytox,16:hour,700:microliter,10:microliter,SC Media,0,Strateos,509:nanometer,488:nanometer,0.274,timeseries,[],lb,1,True,True,True,False,True,https://hub.sd2e.org/user/sd2e/design/B_subtilis_WT_JH642/1,30,8,False,0,CP_NovelChassis_OR_Circuit_Cycle0_0_e6884bba-1bc4-11ea-9b49-0242ac110002,CP_NovelChassis_OR_Circuit_Cycle0,PLATE_READER,0.5,0,ct1drnsvwmpex75
1,3,1,0.0,10:microliter,"{'containerId': 'ct1dt583rfgjx8p', 'wellIndex': 0}",select_cols,Water,SC Media,1000:microliter,same-induction-plate,30:minute,8:hour,100:microliter,None,0:microliter,100:microliter,None,0:microliter,Sytox,16:hour,700:microliter,10:microliter,SC Media,0,Strateos,509:nanometer,488:nanometer,0.274,timeseries,[],lb,1,True,True,True,False,True,https://hub.sd2e.org/user/sd2e/design/B_subtilis_WT_JH642/1,30,8,False,0,CP_NovelChassis_OR_Circuit_Cycle0_0_e6884bba-1bc4-11ea-9b49-0242ac110002,CP_NovelChassis_OR_Circuit_Cycle0,PLATE_READER,1.0,0,ct1drnsvwmpex75
2,3,1,0.0,10:microliter,"{'containerId': 'ct1dt583rfgjx8p', 'wellIndex': 0}",select_cols,Water,SC Media,1000:microliter,same-induction-plate,30:minute,8:hour,100:microliter,None,0:microliter,100:microliter,None,0:microliter,Sytox,16:hour,700:microliter,10:microliter,SC Media,0,Strateos,509:nanometer,488:nanometer,0.274,timeseries,[],lb,1,True,True,True,False,True,https://hub.sd2e.org/user/sd2e/design/B_subtilis_WT_JH642/1,30,8,False,0,CP_NovelChassis_OR_Circuit_Cycle0_0_e6884bba-1bc4-11ea-9b49-0242ac110002,CP_NovelChassis_OR_Circuit_Cycle0,PLATE_READER,1.5,0,ct1drnsvwmpex75
3,3,1,0.0,10:microliter,"{'containerId': 'ct1dt583rfgjx8p', 'wellIndex': 0}",select_cols,Water,SC Media,1000:microliter,same-induction-plate,30:minute,8:hour,100:microliter,None,0:microliter,100:microliter,None,0:microliter,Sytox,16:hour,700:microliter,10:microliter,SC Media,0,Strateos,509:nanometer,488:nanometer,0.274,timeseries,[],lb,1,True,True,True,False,True,https://hub.sd2e.org/user/sd2e/design/B_subtilis_WT_JH642/1,30,8,False,0,CP_NovelChassis_OR_Circuit_Cycle0_0_e6884bba-1bc4-11ea-9b49-0242ac110002,CP_NovelChassis_OR_Circuit_Cycle0,PLATE_READER,2.0,0,ct1drnsvwmpex75
4,3,1,0.0,10:microliter,"{'containerId': 'ct1dt583rfgjx8p', 'wellIndex': 0}",select_cols,Water,SC Media,1000:microliter,same-induction-plate,30:minute,8:hour,100:microliter,None,0:microliter,100:microliter,None,0:microliter,Sytox,16:hour,700:microliter,10:microliter,SC Media,0,Strateos,509:nanometer,488:nanometer,0.274,timeseries,[],lb,1,True,True,True,False,True,https://hub.sd2e.org/user/sd2e/design/B_subtilis_WT_JH

In [55]:
#df.lab_id
#f.loc[df.batch==1] #.drop(columns='id').drop_duplicates()
#df.drop(columns='timepoint').drop_duplicates().groupby([  "strain"]).agg(len)
#df.drop(columns=['timepoint', 'measurement_type']).drop_duplicates().groupby([ 'strain',  'beta_estradiol_concentration', 'batch']).agg(len).reset_index()
df.drop(columns=['timepoint', 'measurement_type']).drop_duplicates().groupby([ 'strain',  'iptg', 'cuminic_acid', 'batch']).agg(len).reset_index()

#df.loc[df['beta_estradiol_concentration'].isna()].strain.value_counts()
#df.strain.unique()
#df.lab_id.unique()
#df.measurement_type.unique()
#df.columns
#len(df.drop(columns=['timepoint', 'measurement_type']).drop_duplicates().loc[df.strain == 'UWBF_24926'])
#len(df.loc[df.strain == 'UWBF_24952'])

,strain,iptg,cuminic_acid,batch,colony,column_id,induction_info.induction_reagents.ind_cult_vol,induction_info.induction_reagents.inducer,induction_info.induction_reagents.inducer_layout.value,induction_info.induction_reagents.inducer_solvent,induction_info.induction_reagents.induction_base_media,induction_info.induction_reagents.media_vol,induction_info.induction_time.induction_plate_info,induction_info.induction_time.sampling_time,induction_info.induction_time.total_induction_time,induction_info.sampling_info.flow_cult_vol,induction_info.sampling_info.flow_solv,induction_info.sampling_info.flow_solv_vol,induction_info.sampling_info.read_cult_vol,induction_info.sampling_info.read_solv,induction_info.sampling_info.read_solv_vol,induction_info.sampling_info.stain_before_flow,inoc_info.inc_time_1,inoc_info.inoc_media_vol,inoc_info.inoc_vol,inoc_info.inoculation_media,lab,plate_reader_info.fluor_em,plate_reader_info.fluor_ex,plate_reader_info.gain,protocol,recovery_info,recovery_media,replicate,run_info.only_endpoint_flow,run_info.read_each_induction,run_info.read_each_recovery,run_info.save_for_rnaseq,run_info.skip_first_flow,temperature,validate_samples,id,experiment_id,output_id,container
0,https://hub.sd2e.org/user/sd2e/design/B_subtilis_GFP_max/1,0,0.0,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1,https://hub.sd2e.org/user/sd2e/design/B_subtilis_GFP_max/1,0,0.0,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,https://hub.sd2e.org/user/sd2e/design/B_subtilis_GFP_max/1,0,0.0,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
3,https://hub.sd2e.org/user/sd2e/design/B_subtilis_GFP_max/1,0,0.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
4,https://hub.sd2e.org/user/sd2e/design/B_subtilis_GFP_max/1,0,0.1,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,https://hub.sd2e.org/user/sd2e/design/B_subtilis_WT_PY79/1,1,0.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
108,https://hub.sd2e.org/user/sd2e/design/B_subtilis_WT_PY79/1,1,0.1,0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
109,https://hub.sd2e.org/user/sd2e/design/B_subtilis_WT_PY79/1,1,0.1,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
110,https://hub.sd2e.org/user/sd2e/design/B_subtilis_WT_PY79/1,1,0.1,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3.0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


In [68]:
#df.loc[(df.timepoint==8.0) & (df.iptg == 0) & (df.cuminic_acid == 0) & (df.temperature ==30) & (df.colony==1) & (df.strain == "https://hub.sd2e.org/user/sd2e/design/B_subtilis_GFP_min/1")]
df.timepoint.value_counts()

24.0    1344
12.0    1344
8.0     1344
7.5     672 
3.5     672 
1.0     672 
1.5     672 
2.0     672 
2.5     672 
3.0     672 
5.0     672 
4.0     672 
6.5     672 
6.0     672 
7.0     672 
4.5     672 
5.5     672 
0.5     672 
Name: timepoint, dtype: int64

In [26]:
from pysmt.shortcuts import Symbol, And, Not, is_sat, get_model

varA = Symbol("A") # Default type is Boolean
varB = Symbol("B")
f = And(varA, Not(varB))
model = get_model(f)
print(model)

A := True
B := False


In [19]:
from pysd2cat.analysis.plate_layout import solve

from pysmt.shortcuts import Symbol, And, Or, Not, Implies, Equals, Iff, is_sat, get_model, GT, GE, LT, Int, String
from pysmt.typing import INT, StringType


## Symbols denoting the samples
samples = ["x1", "x2", "x3", "x4"]

## Factor definitions
factors = { 
    "media" : {
        "domain" : ['m1', "m2"],
        "ftype" : "batch"
    }, 
    "strain" : {
        "domain" : [ "s1", "s2"],
        "ftype" : "sample"
    },
    "temperature" : {
        "domain" : [ "30", "35"],
        "ftype" : "experiment"
    },
    "measurement_type" : {
        "domain" : [ "PLATE_READER", "FLOW"],
        "ftype" : "shadow"
    },
    "inducer" : {
        "domain" : [ "0", "1"],
        "ftype" : "column"
    }
}


containers = { "c1" : {
                    "aliquots" : {
                        "a1" : {
                            "strain" : "s1"
                        },
                        "a2" : {
                            "strain" : "s2"
                        } 
                    },
                    "columns" : { "col1" : ["a1", "a2"] }
                } 
             }



## Requirement for the samples
requirements = [
    {
        "factors" : [
            {
                "factor" : "media", "values" : [ "m2" ]
            },
            {
                "factor" : "strain", "values" : [ "s1", "s2" ]
            },
            {
                "factor" : "temperature", "values" : [ "30" ]
            },
            {
                "factor" : "measurement_type", "values" : [ "PLATE_READER", "FLOW" ]
            },
            {
                "factor" : "inducer", "values" : [  "1" ]
            }
        ]
    }
]

inputs = {
    "samples" : samples,
    "factors" : factors,
    "containers" : containers,
    "requirements" : requirements
}


model = solve(inputs)
if model:
    print(model)
else:
    print("No solution found")

'media(x1)' := 1
'inducer(x4)' := 1
'strain(x1)' := 1
'inducer(x2)' := 1
'measurement_type(x4)' := 1
'inducer(x3)' := 1
'inducer(x1)' := 1
'strain(x4)' := 1
'temperature(x4)' := 0
'media(x4)' := 1
'media(x3)' := 1
'temperature(x3)' := 0
'strain(x2)' := 0
'measurement_type(x1)' := 0
'temperature(x1)' := 0
'measurement_type(x3)' := 1
'temperature(x2)' := 0
media_c1_batch := 1
'measurement_type(x2)' := 0
'strain(x3)' := 0
tau_x1 := 1
tau_x3 := 0
'media(x2)' := 1
temperature_exp := 0
tau_x2 := 0
tau_x4 := 1


In [125]:
def factor_cross_product(factors, cross_product):
    if len(factors) == 0:
        return cross_product
    else:
        result = []
        factor = factors.pop(0)
        for elt in cross_product:
            for value in factor['values']:
                expansion = elt.copy()
                expansion.update({factor['factor'] : value})
                result.append(expansion)
        return factor_cross_product(factors, result)
    

def expand_requirement(requirement):
    factors = requirement['factors']
    expansion = factor_cross_product(factors.copy(), [{}])
    return expansion

expand_requirement(requirements[0])

[{'media': 'm2', 'strain': 's1', 'temperature': '30'},
 {'media': 'm2', 'strain': 's1', 'temperature': '35'},
 {'media': 'm2', 'strain': 's2', 'temperature': '30'},
 {'media': 'm2', 'strain': 's2', 'temperature': '35'}]

In [83]:
from functools import reduce  
import operator

class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

def getFromDict(dataDict, mapList):
    return reduce(operator.getitem, mapList, dataDict)

def setInDict(dataDict, mapList, value):
    getFromDict(dataDict, mapList[:-1])[mapList[-1]] = value
        
def make_entry(av_dict, key, value):
    """
    Helper function to make turn dot-ified key string into a list of keys
    corresponding to nested dicts and assign value to resulting nesting.  If
    path indicated by keys does not exist, then create it.
    """
    key_list = key.split('.')
    setInDict(av_dict, key_list, value)

d=AutoVivification()
make_entry(d, "foo[0]", "bar")
d

{'foo[0]': 'bar'}